# Resumo do código

>### Entra: StocksDelta2022  
>### Sai: 2_Delta_2022Id
---
O objectivo é receber dados dos ficheiros totais da Delta e devolver um ficheiro com os dados dos produtos e das lojas relevantes para o estudo da evolução do padrão de vendas (dias homólogos do ano anterior). 

Devolve um ficheiro que pode entrar no código 2 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados da Delta Todos__: Lojas e produtos todos
> - Todas as métricas possíveis de retirar só dos dados Delta


- Outputs

> **2_Delta_2022Id**:
> - Ficheiro com todas as métricas nas datas homólogas correspondentes aos dias em que ocorreram as missões


In [36]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

# Ler ficheiro com tudo e os específicos

- Tudo

In [41]:
%%time
dfTudo=pd.read_csv('D:\\B&N Dados\\Delta\\Stocks2022\\StocksDelta2022.csv')

CPU times: total: 14.2 s
Wall time: 21.1 s


> Passar para datetime

In [42]:
dfTudo = dfTudo.rename(columns={'Dia_Actual': 'SELLOUT', "1_Dia_Antes":"SELLOUT_1_Dias_Antes"})
dfTudo['DATA']= pd.to_datetime(dfTudo['DATA'], format='%Y-%m-%d')

# Encontrar os nomes no dataFrame de tudo

In [43]:
#dfTudo['DESC_ARTIGO'].sort_values().unique()


#Lojas:
                  #Reposição=Sim
#CNT AMADORA
#CNT BRAGA
#CNT CASCAIS
#CNT COIMBRASHOPPING
#CNT MATOSINHOS
                  #Reposição=Não
#CNT LEIRIA
#CNT SANTAREM
#CNT VIANA
#MDL T. VEDRAS                                              
#CNT PORTIMÃO

#CAFÉ DELTA Q QALIDUS 80CAP
#CAFÉ DELTA Q MYTHIQ 80CAP
#CAFÉ DELTA Q QALIDUS 10CAP
#CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G
#CAFÉ BELLISSIMO INTENSO 200GR
#CAFÉ DELTA RITUAL MU 220G
#CAFÉ SOLÚVEL DELTA FRASCO 200G
#CAFÉ SOLÚVEL DELTA FRASCO 100G
#CEVADA SOLÚVEL DELTA FRASCO 200G

#Produto novo

# Fazer subsets da dataFrame total

> - Por loja e produto

In [44]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\lojas.csv', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

# Definir o dataframe final apenas tendo em conta as lojas e os produtos requeridos
dfFinal = dfTudo[dfTudo['STORE'].isin(lojas) & dfTudo['DESC_ARTIGO'].isin(produtos)].copy()

> - Datas homólogas

In [46]:
dfFinal=dfFinal.loc[(dfFinal['DATA'] >= '2022-04-27') & (dfFinal['DATA'] <= '2022-04-29')].copy()

# Temos
- dfFinal com dados Delta nos dias específicos

# Queremos
- id

 - Criar um **índice** para cada dia

In [48]:
df=dfFinal.copy()

# Extract the day and month components from the 'datetime' column
df['day'] = pd.to_datetime(df['DATA']).dt.day
df['month'] = pd.to_datetime(df['DATA']).dt.month

# Create a new column combining day and month
df['day_month'] = df['day'].astype(str) + '-' + df['month'].astype(str)

# Assign unique IDs to each unique day-month combination
df['id'], _ = pd.factorize(df['day_month'])

df = df.drop(columns=['day', 'month', 'day_month'])

# Temos:
- Uma base apenas com os fins de semana em causa com todas as métricas que estavam até agora mais a **Semana**, o **Dia** e o **Id** para poder ser mergido.

# Escrever

In [50]:
escrever_excel(df, "2_Delta_2022Id")

# Média de Stock por loja e produto

In [19]:
file1MedSto = dfFdsPiloto.groupby(['STORE_NAME', 'DESC_ARTIGO'])['SOH'].mean().reset_index()
file1StdSto = dfFdsPiloto.groupby(['STORE_NAME', 'DESC_ARTIGO'])['SOH'].std().reset_index()

stocks1 = pd.merge(file1MedSto, file1StdSto[['STORE_NAME', 'DESC_ARTIGO', 'SOH']], on=['STORE_NAME', 'DESC_ARTIGO'])
stocks1 = stocks1.rename(columns={'SOH_x': 'Stock_médio_Fds', 'SOH_y': 'sdSto'})